In [42]:
import tqdm as notebook_tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from sacrebleu.metrics import BLEU
from tqdm import tqdm
import sentencepiece as spm
import ctranslate2

In [65]:
# %cd /home/rana/Projects/zindi
%cd /root/zindi/
import yaml
with open('common/config.yaml', 'r') as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)

checkpoint = config.get('checkpoint')
new_model_path=config.get('new_model_path')+checkpoint
ct_model_path=config.get('ct_model_path')+checkpoint

/root/zindi


In [68]:
### Eval ct model

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

sp_source_model = spm.SentencePieceProcessor(model_file=ct_model_path+'/source.spm')
sp_target_model = spm.SentencePieceProcessor(model_file=ct_model_path+'/target.spm')


def translate(translator, text):
    tokens = sp_source_model.encode(text, out_type=str)
    # print(tokens)
    tokens = ['dyu'] + tokens + ["</s>"] +['fra']
    try:
        results = translator.translate_batch([tokens])
        # The translated results are token strings, so we need to convert them to IDs before decoding
        translations = []
        for translation in results:
            decoded_text = sp_target_model.decode(translation.hypotheses[0])
            translations.append(decoded_text)
    except Exception as e:
        print(f"Translation error: {e}")
        translations = [""]  # Return empty string if translation fails
    return translations

def calculate_bleu(references, hypotheses):
    bleu = BLEU()
    return bleu.corpus_score(hypotheses, [references]).score

def validate_model(model_path, dataset_name, split):
    # Load CTranslate2 model
    translator = ctranslate2.Translator(model_path, device=device)
    # Load dataset
    dataset = load_dataset(dataset_name, split=split)
    # Translate and calculate BLEU score
    hypotheses = []
    references = []

    for batch in tqdm(dataset, desc="Translating"):
        source_text = batch['translation']['dyu']  # Adjust this based on your dataset's column names
        reference = batch['translation']['fr']  # Adjust this based on your dataset's column names
        
        # translation = translate(translator, tokenizer, source_text, device)[0]
        # tokenizer=None
        translation = translate(translator, source_text)[0]
        hypotheses.append(translation)
        references.append(reference)
        # print(translation, "----", reference)
        # break

    bleu_score = calculate_bleu(references, hypotheses)
    print(f"BLEU Score: {bleu_score:.2f}")

validate_model(ct_model_path, "uvci/Koumankan_mt_dyu_fr", "validation")

Using device: cuda


Translating:   0%|          | 6/1471 [00:00<00:29, 50.47it/s]

Comment t ⁇ appelles-tu ? ---- Tu portes un nom de fantaisie.
Les trois premiers. ---- Trois points d’avance.
L'écoulement de soleil ---- Le soleil s’est couché.
À propos de l'une ---- Mêmes mouvements.
Nous n'avons pas de nourriture. ---- Je n’ai pas encore déjeuné.
" Elle m'a ouvert la bouche. ---- Arrêtez de vous moquer de moi.
Je suis américain. ---- Je suis Américain.
Monsieur Dextrine, monsieur le député ---- Merci, monsieur le député
Il n'a jamais connu son père ---- Jean n’a pas connu son père.
Nous devons le mettre à l'intérieur. ---- Il faudra rentrer dedans.
0 ---- Neuf heures dix.


Translating:   1%|          | 18/1471 [00:00<00:27, 53.27it/s]

Ça va faire peur. ---- Cela serait épouvantable.
" Qu'est-ce que je vais faire ? ---- Comment je fais pour y aller ?
Ma mère me l'a dit de nouveau. ---- Ma mère me le répète.
Certains sont très tôt installés. ---- D’autres maisons plus jaunes.
Quel est l'hommage du sang ? ---- Ce coureur était le combien ?
J'ai pris les nombres suivants. ---- J'étais à deux pas.
Je vous trouve à la porte. ---- J'ouvre la porte.
Qu ⁇ est-ce que vous avez fait ? ---- Et pourquoi donc ?
Vous savez que vous n'avez pas confiance en vous ---- Que vous soyez retenue ou non.
Regard sur l'homme de Canaan ---- Hum! fit le Canadien.
Il répondit à la question. ---- Jusqu'à en tomber amoureuse...
deux portes. ---- C'est deux francs !


...

Translating:   4%|▍         | 58/1471 [00:01<00:23, 60.21it/s]

Le langage de l'avoine ---- Vous parlez français ?
vingt et une chiennes. ---- vingt et une chiennes.
Deux femmes ont été tuées. ---- Elle jouait avec elle.
Un oiseau qui se tient sur nos genoux. ---- Je suis soumise, testez-moi!
Il s'est bien amusé ! ---- Ah ! c’est très fort !
Maintenant, c ⁇ est Sarah. ---- Paris, de nos jours.
C ⁇ est un professeur. ---- C’est votre entraineur ?
Avez-vous déjà eu l ⁇ occasion d ⁇ apprendre ? ---- Tu apprends bien tes leçons.
J ⁇ ai toujours aimé. ---- Je suis sûr que j’aurais aimé.
0 ---- Il est malheureux
C ⁇ est le sang de la même journée. ---- Combien ça coûte par jour ?
0 ---- Voilà qui est bien
Rue Achiten, Aurillon ---- L'action se déroule à Palerme.


Translating:   5%|▍         | 69/1471 [00:01<00:24, 56.94it/s]

Je ne sais pas ce qu ⁇ il veut. ---- je ne sais pas ce qu'il veut.
0 ---- Peut-être que j’irai.
- Vous comprenez ? ---- Vous conprenez ?
Je ne l'ai pas vue. ---- Je ne les ai pas trouvés.
La voix est trop haute. ---- un longue lettre.
0 ---- Un pressoir daté du le jouxte.
La paix dans le monde ? ---- mille cent onze-deux et L.


KeyboardInterrupt: 

In [67]:
validate_model(ct_model_path, "uvci/Koumankan_mt_dyu_fr", "train")

Translating: 100%|██████████| 8065/8065 [02:58<00:00, 45.17it/s]


BLEU Score: 7.67


In [ ]:
### Eval original model 

# def translate(model, tokenizer, text, device):
#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=150).to(device)
#     translated = model.generate(**inputs)
#     return tokenizer.batch_decode(translated, skip_special_tokens=True)

# def calculate_bleu(references, hypotheses):
#     bleu = BLEU()
#     return bleu.corpus_score(hypotheses, [references]).score

# def validate_model(model_name, dataset_name, split):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"Using device: {device}")

#     # Load model and tokenizer
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

#     # Load dataset
#     dataset = load_dataset(dataset_name, split=split)

#     # Translate and calculate BLEU score
#     hypotheses = []
#     references = []

#     for batch in tqdm(dataset, desc="Translating"):
#         source_text = batch['translation']['dyu']  # Adjust this based on your dataset's column names
#         reference = batch['translation']['fr']  # Adjust this based on your dataset's column names
        
#         translation = translate(model, tokenizer, source_text, device)[0]
#         # print(reference)
#         # print(translation)
        
#         hypotheses.append(translation)
#         references.append(reference)
#         # break

#     bleu_score = calculate_bleu(references, hypotheses)
#     print(f"BLEU Score: {bleu_score:.2f}")

# # Run validation with GPU
# # validate_model(model_checkpoint, "uvci/Koumankan_mt_dyu_fr", use_gpu=True)

# # Run validation without GPU
# validate_model(new_model_path, "uvci/Koumankan_mt_dyu_fr", "validation")